In [100]:
%matplotlib inline
from matplotlib import pyplot as plt
import operator
from scipy.spatial.distance import cdist
from typing import Callable, Tuple
from scipy.spatial.distance import euclidean
import numpy as np
from collections import deque


class STFrechet:
    def __init__(self, a: np.ndarray, b: np.ndarray, dist: Callable[[np.ndarray, np.ndarray], float]):
        self.a = a
        self.b = b
        self.dist_func = dist
        self.dist_matrix = cdist(a, b, self.dist_func)
        self.cost, self.C = self.full()
        self.path = []

    def find_path(self):
        t0 = self.a
        t1 = self.b
        C = self.C
        n0 = len(t0)
        n1 = len(t1)
        if C[n0, n1] == self.cost:
            self.path = sorted(map(tuple, np.argwhere(C == self.cost).tolist()), key=operator.itemgetter(0, 1))
            return self.path
        queue = deque([(n0, n1)])
        path = {(n0, n1)}

        while queue:
            i, j = queue.popleft()
            if i == j == 1:
                continue
            candidates = [
                C[i - 1, j - 1],
                C[i, j - 1],
                C[i - 1, j],
            ]
            king = min(candidates)
            for x, y in [(i - 1, j), (i - 1, j), (i - 1, j - 1)]:
                if C[x, y] == king:
                    queue.append((x, y))
                    path.add((x, y))
        self.path = sorted(path, key=operator.itemgetter(0, 1))
        return self.path

    def move(self, a_remove: int = 0, b_remove: int = 0, a_add: np.ndarray = None, b_add: np.ndarray = None):
        next_start: Tuple[int, int] = (a_remove + 1, b_remove + 1)

        # if self.C[next_start]!=self.cost:
        if next_start not in self.find_path():
            if a_remove > 0:
                self.a = self.a[a_remove:]
            if b_remove > 0:
                self.b = self.b[b_remove:]
            if a_add is not None:
                self.a = np.vstack([self.a, a_add])
            if b_add is not None:
                self.b = np.vstack([self.b, b_add])

            self.dist_matrix = cdist(self.a, self.b, self.dist_func)

            self.cost, self.C = self.full()

            return self.cost, False
        else:
            if a_add is not None or b_add is not None:
                self.cost = self.add(a_add, b_add)
            next_start: Tuple[int, int] = (a_remove + 1, b_remove + 1)
            # if self.C[next_start]==self.cost:
            if next_start in self.find_path():
                if a_remove > 0 or b_remove > 0:
                    self.cost = self.remove(a_remove, b_remove)
                    return self.cost, True  # indicate hit
            else:
                if a_remove > 0 or b_remove > 0:
                    if a_remove > 0:
                        self.a = self.a[a_remove:]
                    if b_remove > 0:
                        self.b = self.b[b_remove:]
                    self.dist_matrix = cdist(self.a, self.b, self.dist_func)
                    self.cost, self.C = self.full()
                # return and call full in async
                return self.cost, False

    def remove(self, a_remove, b_remove):
        # hit
        print("HIT")
        # frechet need to regenerate through path
        # find path using bfs
        C = self.C

        path = self.path

        idx = path.index((a_remove + 1, b_remove + 1))
        self.C[a_remove + 1, b_remove + 1] = self.dist_matrix[a_remove, b_remove]
        for c in range(idx + 1, len(path)):
            i, j = path[c][0], path[c][1]
            candidates = [
                C[i - 1, j - 1],
                C[i, j - 1],
                C[i - 1, j],
            ]
            king = min(candidates)
            C[i, j] = max(self.dist_matrix[i - 1, j - 1], king)
        n0, n1 = path[-1]
        self.cost = self.C[n0, n1]
        return self.cost

    def add(self, a_add: np.ndarray, b_add: np.ndarray):
        if a_add is not None:
            self.dist_matrix = np.vstack([self.dist_matrix, cdist(a_add, self.b)])
            self.a = np.vstack([self.a, a_add])
        if b_add is not None:
            self.dist_matrix = np.hstack([self.dist_matrix, cdist(self.a, b_add)])
            self.b = np.vstack([self.b, b_add])
        t0 = self.a
        t1 = self.b
        n0 = len(t0)
        n1 = len(t1)
        C = np.zeros((n0 + 1, n1 + 1))
        C[1:, 0] = float('inf')
        C[0, 1:] = float('inf')
        ori0, ori1 = self.C.shape
        C[:ori0, :ori1] = self.C
        for i in np.arange(1, ori0):
            for j in np.arange(ori1, n1 + 1):
                candidates = [
                    C[i - 1, j - 1],
                    C[i, j - 1],
                    C[i - 1, j],
                ]
                king = min(candidates)
                C[i, j] = max(self.dist_matrix[i - 1, j - 1], king)
        for i in np.arange(ori0, n0 + 1):
            for j in np.arange(1, n1 + 1):
                candidates = [
                    C[i - 1, j - 1],
                    C[i, j - 1],
                    C[i - 1, j],
                ]
                king = min(candidates)
                C[i, j] = max(self.dist_matrix[i - 1, j - 1], king)

        dtw = C[n0, n1]
        self.cost, self.C = dtw, C

        # print(f"add: {t5 - t4}")
        return dtw

    def full(self):
        t0 = self.a
        t1 = self.b
        n0 = len(t0)
        n1 = len(t1)
        C = np.zeros((n0 + 1, n1 + 1))
        C[1:, 0] = float('inf')
        C[0, 1:] = float('inf')
        for i in np.arange(n0) + 1:
            for j in np.arange(n1) + 1:
                candidates = [
                    C[i - 1, j - 1],
                    C[i, j - 1],
                    C[i - 1, j],
                ]
                king = min(candidates)
                C[i, j] = max(self.dist_matrix[i - 1, j - 1], king)
        dtw = C[n0, n1]
        # print(f"full: {t5-t4}")
        return dtw, C


a = np.asarray([[197],
                [188],
                [152],
                [144],
                [164]])
b = np.asarray([[43],
                [45],
                [137],
                [14],
                [139]])

truth = STFrechet(a[2:], b[2:], euclidean)
our = STFrechet(a[:3], b[:3], euclidean)
our.move(2, 2, a_add=a[3:], b_add=b[3:])
print(truth.cost, our.cost)
print(our.dist_matrix)
print(our.C)
print(truth.cost == our.cost)

HIT
130.0 130.0
[[154. 152.  60. 183.  58.]
 [145. 143.  51. 174.  49.]
 [109. 107.  15. 138.  13.]
 [101.  99.   7. 130.   5.]
 [121. 119.  27. 150.  25.]]
[[  0.  inf  inf  inf  inf  inf]
 [ inf 154. 154. 154. 183. 183.]
 [ inf 154. 154. 154. 174. 174.]
 [ inf 154. 154.  15. 138. 138.]
 [ inf 154. 154.  15. 130. 130.]
 [ inf 154. 154.  27. 150. 130.]]
True


In [69]:
from scipy.spatial.distance import euclidean
import time
from tqdm.autonotebook import tqdm
import numpy as np


def discret_frechet(t0, t1):
    """
    Usage
    -----
    Compute the discret frechet distance between trajectories P and Q
    Parameters
    ----------
    param t0 : px2 numpy_array, Trajectory t0
    param t1 : qx2 numpy_array, Trajectory t1
    Returns
    -------
    frech : float, the discret frechet distance between trajectories t0 and t1
    """
    n0 = len(t0)
    n1 = len(t1)
    C = np.zeros((n0 + 1, n1 + 1))
    C[1:, 0] = float('inf')
    C[0, 1:] = float('inf')
    for i in np.arange(n0) + 1:
        for j in np.arange(n1) + 1:
            C[i, j] = max(euclidean(t0[i - 1], t1[j - 1]), min(C[i, j - 1], C[i - 1, j - 1], C[i - 1, j]))
    dtw = C[n0, n1]
    return dtw


DATASET = 20
WINDOW = 700
STEP = 2
loose = 0
truth_t = 0
ours_t = 0
hit_t = 0
hit_c = 0
for _ in tqdm(range(DATASET)):
    a = np.random.randint(0, 200, (WINDOW + STEP, 1))
    b = np.random.randint(0, 200, (WINDOW + STEP, 1))
    t1 = time.perf_counter_ns()
    truth = discret_frechet(a[2:], b[2:])
    t2 = time.perf_counter_ns()
    our = STFrechet(a[:500], b[:500], euclidean)
    t3 = time.perf_counter_ns()
    o, hit = our.move(2, 2, a_add=a[500:], b_add=b[500:])
    t4 = time.perf_counter_ns()
    truth_t += t2 - t1
    ours_t += t4 - t3
    if hit:
        hit_c += 1
        hit_t += t4 - t3
    if (t4 - t3 > t2 - t1):
        loose += 1
    #     print(t4 - t3, t2 - t1)
    if truth != o:
        print(truth.cost)
        print(o)
        print(truth.C)
        print(our.C)
        print(our.dist_matrix)
        print(a)
        print(b)
        assert False
print(truth_t / 1e9 / DATASET, ours_t / 1e9 / DATASET, hit_t / 1e9 / DATASET)
print(loose)

  0%|          | 0/20 [00:00<?, ?it/s]

HIT
HIT
HIT
2.9162559463 2.62279836555 0.14018425175
5


In [163]:
# random_add
from scipy.spatial.distance import euclidean
import time
from tqdm.autonotebook import tqdm
import numpy as np


def discret_frechet(t0, t1):
    """
    Usage
    -----
    Compute the discret frechet distance between trajectories P and Q
    Parameters
    ----------
    param t0 : px2 numpy_array, Trajectory t0
    param t1 : qx2 numpy_array, Trajectory t1
    Returns
    -------
    frech : float, the discret frechet distance between trajectories t0 and t1
    """
    n0 = len(t0)
    n1 = len(t1)
    C = np.zeros((n0 + 1, n1 + 1))
    C[1:, 0] = float('inf')
    C[0, 1:] = float('inf')
    for i in np.arange(n0) + 1:
        for j in np.arange(n1) + 1:
            C[i, j] = max(euclidean(t0[i - 1], t1[j - 1]), min(C[i, j - 1], C[i - 1, j - 1], C[i - 1, j]))
    dtw = C[n0, n1]
    return dtw


DATASET = 20
WINDOW = 5000
STEP = 2
loose = 0
truth_t = 0
ours_t = 0
hit_t = 0
hit_c = 0
for _ in tqdm(range(DATASET)):
    a = np.random.randint(0, 200, (WINDOW + STEP, 1))
    b = np.random.randint(0, 200, (WINDOW + STEP, 1))
    t1 = time.perf_counter_ns()
    truth = discret_frechet(a[2:], b[2:])
    t2 = time.perf_counter_ns()
    our = STFrechet(a[:500], b[:500], euclidean)
    t3 = time.perf_counter_ns()
    o, hit = our.move(2, 2, a_add=a[500:], b_add=b[500:])
    t4 = time.perf_counter_ns()
    truth_t += t2 - t1
    ours_t += t4 - t3
    if hit:
        hit_c += 1
        hit_t += t4 - t3
    if (t4 - t3 > t2 - t1):
        loose += 1
    #     print(t4 - t3, t2 - t1)
    if truth != o:
        print(truth.cost)
        print(o)
        print(truth.C)
        print(our.C)
        print(our.dist_matrix)
        print(a)
        print(b)
        assert False
print(truth_t / 1e9 / DATASET, ours_t / 1e9 / DATASET, hit_t / 1e9 / DATASET)
print(loose)

  0%|          | 0/20 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [25]:
a = np.random.randint(0, 200, (10, 1))
b = np.random.randint(0, 200, (10, 1))
_ = STFrechet(a[:3], b[:3], euclidean)

truth = STFrechet(a[2:], b[2:], euclidean)

our = STFrechet(a[:3], b[:3], euclidean)

%time o = our.move(2, 2, a_add=a[3:], b_add=b[3:])

CPU times: user 656 µs, sys: 0 ns, total: 656 µs
Wall time: 623 µs


In [26]:
test_a = a
test_b = b
print(test_a)
print(test_b)

[[ 85]
 [ 64]
 [100]
 [ 75]
 [107]
 [131]
 [  4]
 [160]
 [109]
 [  3]]
[[139]
 [ 58]
 [150]
 [ 78]
 [100]
 [ 70]
 [  7]
 [184]
 [129]
 [146]]


In [8]:
bench = STFrechet(test_a[2:], test_b[2:], euclidean)

bench_out = STFrechet(test_a[:3], test_b[:3], euclidean)

%time o = bench_out.move(a_add=test_a[3:], b_add=test_b[3:])
bench_out.C

NameError: name 'test_a' is not defined

In [83]:
import pathlib
from tqdm.autonotebook import tqdm

tasks = []


def read_tdrive(fname: pathlib.Path) -> np.ndarray:
    with open(fname, 'r') as f:
        lines = f.readlines()
    res = np.zeros((len(lines), 2))
    for idx, l in enumerate(map(str.strip, lines)):
        p = l.split(",")
        res[idx][0] = float(p[2])
        res[idx][1] = float(p[3])
    return res


for i in tqdm(pathlib.Path("/home/liontao/work/data/tdrive/taxi_log_2008_by_id").glob("*.txt")):
    t = read_tdrive(i)
    if len(t) > 0:
        tasks.append(t)
len(tasks)

0it [00:00, ?it/s]

10336

In [162]:
import random
import math
import pathlib
from tqdm.contrib.concurrent import process_map
import polars as pl
from itertools import combinations

size = 5
WINDOW = 5000
step = 300

rad = math.pi / 180.0
R = 6378137.0


def discret_frechet(t0, t1):
    """
    Usage
    -----
    Compute the discret frechet distance between trajectories P and Q

    Parameters
    ----------
    param t0 : px2 numpy_array, Trajectory t0
    param t1 : qx2 numpy_array, Trajectory t1

    Returns
    -------
    frech : float, the discret frechet distance between trajectories t0 and t1
    """
    n0 = len(t0)
    n1 = len(t1)
    C = np.zeros((n0 + 1, n1 + 1))
    C[1:, 0] = float('inf')
    C[0, 1:] = float('inf')
    for i in range(1, n0 + 1):
        for j in range(1, n1 + 1):
            C[i, j] = max(gpsdist(t0[i - 1], t1[j - 1]), min(C[i, j - 1], C[i - 1, j - 1], C[i - 1, j]))
    dtw = C[n0, n1]
    return dtw


def great_circle_distance_gps(lon1, lat1, lon2, lat2):
    dlat = rad * (lat2 - lat1)
    dlon = rad * (lon2 - lon1)
    a = (math.sin(dlat / 2.0) * math.sin(dlat / 2.0) +
         math.cos(rad * lat1) * math.cos(rad * lat2) *
         math.sin(dlon / 2.0) * math.sin(dlon / 2.0))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c
    return d


def gpsdist(a, b):
    return great_circle_distance_gps(a[0], a[1], b[0], b[1])


def bench_one(task):
    a, b = task
    t1 = time.perf_counter_ns()
    truth = discret_frechet(a[step:step + WINDOW], b[step:step + WINDOW])
    t2 = time.perf_counter_ns()
    our = STFrechet(a[:WINDOW], b[:WINDOW], gpsdist)
    t3 = time.perf_counter_ns()
    _, hit = our.move(step, step, a[WINDOW:WINDOW + step], b[WINDOW:WINDOW + step])
    t4 = time.perf_counter_ns()
    return {"our": t4 - t3, "truth": t2 - t1, "hit": hit}


# bench_one((tasks[0],tasks[1]))
# candidates = random.sample(tasks, size)
candidates = list(filter(lambda x:len(x)>6000,tasks))[:size]
data = process_map(bench_one, combinations(candidates, 2), max_workers=10)
stat = pl.from_dicts(data)
print(stat.mean())
print(stat.filter(pl.col("hit") == True).mean())


0it [00:00, ?it/s]

shape: (1, 3)
┌───────────┬───────────┬─────┐
│ our       ┆ truth     ┆ hit │
│ ---       ┆ ---       ┆ --- │
│ f64       ┆ f64       ┆ f64 │
╞═══════════╪═══════════╪═════╡
│ 1.7879e11 ┆ 1.5529e11 ┆ 0.0 │
└───────────┴───────────┴─────┘
shape: (1, 3)
┌──────┬───────┬──────┐
│ our  ┆ truth ┆ hit  │
│ ---  ┆ ---   ┆ ---  │
│ f64  ┆ f64   ┆ f64  │
╞══════╪═══════╪══════╡
│ null ┆ null  ┆ null │
└──────┴───────┴──────┘


In [48]:
%time discret_frechet(tasks[3][1:WINDOW], tasks[1][1:WINDOW])
our = STFrechet(tasks[3][:WINDOW], tasks[1][:WINDOW], gpsdist)
%time our.move(1, 1)

CPU times: user 804 ms, sys: 0 ns, total: 804 ms
Wall time: 815 ms
CPU times: user 836 ms, sys: 0 ns, total: 836 ms
Wall time: 836 ms


14024.432139196728

In [38]:
def discret_frechet(t0, t1):
    """
    Usage
    -----
    Compute the discret frechet distance between trajectories P and Q
    Parameters
    ----------
    param t0 : px2 numpy_array, Trajectory t0
    param t1 : qx2 numpy_array, Trajectory t1
    Returns
    -------
    frech : float, the discret frechet distance between trajectories t0 and t1
    """
    n0 = len(t0)
    n1 = len(t1)
    C = np.zeros((n0 + 1, n1 + 1))
    C[1:, 0] = float('inf')
    C[0, 1:] = float('inf')
    for i in np.arange(n0) + 1:
        for j in np.arange(n1) + 1:
            C[i, j] = max(euclidean(t0[i - 1], t1[j - 1]), min(C[i, j - 1], C[i - 1, j - 1], C[i - 1, j]))
    dtw = C[n0, n1]
    return dtw


a = np.asarray([[197],
                [188],
                [152],
                [144],
                [164]])
b = np.asarray([[43],
                [45],
                [137],
                [14],
                [139]])
%time _=discret_frechet(a[2:], b[2:])
hit_out = STFrechet(a[:3], b[:3], euclidean)
%time hit_out.move(2, 2, a[3:], b[3:])

CPU times: user 616 µs, sys: 4 µs, total: 620 µs
Wall time: 454 µs
CPU times: user 391 µs, sys: 3 µs, total: 394 µs
Wall time: 370 µs


130.0